## LangChain's Components

### Models and Prompts

In [ ]:
from langchain.llms import OpenAI
import os

os.environ["OPENAP_API_KEY"]

llm = openAI()
print(llm('tell me a joke'))

In [ ]:
from langchain import PromptTemplate

template = """Sentence: {sentence}
Translation in {language}:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

print(prompt.format(sentence = "the cat is on the table", language="spanish"))

### Data Connections

#### Document loaders

In [ ]:
import csv

# Sample data
data = [
    ['Name', 'Age', 'City'],
    ['John', 25, 'New York'],
    ['Emily', 28, 'Los Angeles'],
    ['Michael', 22, 'Chicago']
]

# File name
file_name = 'sample.csv'

# Write data to csv file
with open(file_name, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(data)

print(f'Sample CSV file "{file_name}" generated and saved.')

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='sample.csv')
data = loader.load()
print(data)